In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Open Pro-Israel Contributions Data

In [4]:
representatives_df = pd.read_csv(
    "data/Money from Pro-Israel to US Representatives, 1990-2024.csv"
)

In [5]:
representatives_df["Amount"] = representatives_df["Amount"].str[1:].astype(int)
representatives_df = (
    representatives_df.groupby("State")["Amount"]
    .sum()
    .reset_index(name="REPRESENTATIVE")
)
representatives_df = representatives_df.rename(columns={"State": "NAME"})

In [6]:
senate_df = pd.read_csv("data/Money from Pro-Israel to US Senators, 1990-2024.csv")

In [7]:
senate_df["Amount"] = senate_df["Amount"].str[1:].astype(int)
senate_df = senate_df.groupby("State")["Amount"].sum().reset_index(name="SENATE")
senate_df = senate_df.rename(columns={"State": "NAME"})

In [8]:
congress_df = senate_df.merge(representatives_df, on="NAME", how="inner")
congress_df["SUM"] = congress_df["REPRESENTATIVE"] + congress_df["SENATE"]

## Merge Data

In [9]:
congress_gdf = state_gdf.merge(congress_df, on="NAME", how="inner")

In [10]:
congress_gdf = congress_gdf.to_crs(9311)
congress_gdf.to_file("data/Pro_Israel_Contributions_Per_State.gpkg")

In [11]:
congress_gdf["SENATE"].sum()

np.int64(98155627)

In [12]:
congress_gdf["REPRESENTATIVE"].sum()

np.int64(177517895)